In [1]:
# import 
import pickle
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,roc_auc_score,roc_curve
import warnings
warnings.filterwarnings('ignore')

!pwd
!ls -a ./raw

/Users/hobee/PythonScript/CCF-BDCI-2022-FXFZ
.                           dataNoLabel.csv
..                          dataTrain.csv
dataA.csv                   submit_example_A.csv
dataB-209019.csv            submit_example_B-560036.csv


In [3]:
# load the data
semi_data = pd.read_csv("./cache/semi_label_data.csv")
raw_data = pd.read_csv("./cache/label_data.csv")
label_data = pd.read_csv("./cache/label.csv")
A_rank_data = pd.read_csv("./cache/no_label_data.csv")

data = semi_data.copy().append(A_rank_data,ignore_index=True)

data.describe()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f42_log_f43_log,f42_log_f44_log,f42_log_f45_log,f42_log_f46_log,f43_log_f44_log,f43_log_f45_log,f43_log_f46_log,f44_log_f45_log,f44_log_f46_log,f45_log_f46_log
count,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,...,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000,89742.000000
mean,0.343986,0.704687,0.975597,6.755566,20.369994,41.299392,30.071672,9.935281,1.771946,2.643545,...,180.985024,181.493426,183.844822,184.809320,8019.629415,8021.980812,8022.945310,13105.999443,13106.963941,36620.926188
std,0.475039,0.456186,0.816532,25.000288,56.536499,100.426553,93.881074,58.024913,21.693476,27.187064,...,2676.772823,2676.789162,2676.810859,2676.813922,14028.286462,14027.862063,14027.878266,17919.606513,17919.695813,22546.846463
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,3.000000,0.000000,2.000000,3.000000,2.000000,3.000000,20003.000000
50%,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,4.000000,5.000000,0.000000,5.000000,6.000000,5.000000,6.000000,40005.000000
75%,1.000000,1.000000,2.000000,0.000000,0.000000,36.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,5.000000,6.000000,10003.000000,10006.000000,10007.000000,20007.000000,20008.000000,50007.000000
max,1.000000,1.000000,2.000000,450.000000,1800.000000,3231.000000,828.000000,738.000000,522.000000,729.000000,...,60005.000000,60007.000000,60008.000000,60009.000000,90010.000000,90011.000000,90011.000000,100010.000000,100011.000000,110011.000000


In [4]:
# load the model
model_file_path = "./models/stack_model.pkl"
model = pickle.load(open(model_file_path, "rb"))

In [5]:
def train_model(x,y, model="PlaceHolder"):
    x_train,x_test,y_train,y_test = train_test_split(x, y, test_size = 0.3,random_state = 33)
    ### 训练模型
    if model == "PlaceHolder":
        model = xgb.XGBClassifier(learning_rate=0.1,
                      n_estimators=100,            # 树的个数--100棵树建立xgboost
                      max_depth=6,                 # 树的深度
                      min_child_weight = 1,        # 叶子节点最小权重
                      gamma=0.,                    # 惩罚项中叶子结点个数前的参数
                      subsample=0.8,               # 随机选择80%样本建立决策树
                      # colsample_btree=0.8,         # 随机选择80%特征建立决策树
                      objective='binary:logistic', # 指定损失函数
                      scale_pos_weight=1,          # 解决样本个数不平衡的问题
                      random_state=27              # 随机数
                      )

    # 拟合
    # model.fit(x_train, y_train, eval_set = [(x_test,y_test)], eval_metric = "auc", early_stopping_rounds = 10,verbose = True)
    model.fit(x_train, y_train)
    
    return model,x_train,x_test,y_train,y_test

In [6]:
# predict
pred_prob_A = model.predict_proba(data)

In [7]:
# 思路 循环训练 每次将label为概率大于90%的加入训练集重新训练 直到结果无法提升
scope = 0.95
last_auc = 0
epoch = 1
print(data.shape)
print(raw_data.shape)
while True:
    pb = model.predict_proba(data)
    pos_item = data[pb[:,0] > scope]
    neg_item = data[pb[:,1] > scope]
    raw_data = raw_data.append(pos_item,ignore_index=True)
    raw_data = raw_data.append(neg_item,ignore_index=True)
    label_data = label_data.append(
        pd.DataFrame({"label": [1 for i in range(pos_item.shape[0])]}),
        ignore_index=True
    )
    label_data = label_data.append(
        pd.DataFrame({"label": [1 for i in range(neg_item.shape[0])]}),
        ignore_index=True
    )
    data = data.drop(pos_item.index,inplace=False)
    data = data.drop(neg_item.index,inplace=False)
    model,x_train,x_test,y_train,y_test = train_model(raw_data,label_data,model)
    if epoch != 1:
        last_auc = auc
    auc = roc_auc_score(y_test, model.predict_proba(x_test)[:,1])
    delta = auc - last_auc
    print(f"epoch:{epoch},auc:{auc},last_auc:{last_auc},dalta:{auc - last_auc}")
    print(data.shape)
    print(raw_data.shape)
    epoch += 1
    if abs(delta) < 0.0001 or epoch == 30:
        print(f"stop at epoch:{epoch}!")
        break

model


(89742, 1129)
(50000, 1129)
epoch:1,auc:0.834841631147561,last_auc:0,dalta:0.834841631147561
(67330, 1129)
(72412, 1129)
epoch:2,auc:0.8351588163198651,last_auc:0.834841631147561,dalta:0.0003171851723040575
(67330, 1129)
(72412, 1129)
epoch:3,auc:0.8348323534526361,last_auc:0.8351588163198651,dalta:-0.00032646286722903817
(67330, 1129)
(72412, 1129)
epoch:4,auc:0.8345791491650397,last_auc:0.8348323534526361,dalta:-0.0002532042875963425
(67330, 1129)
(72412, 1129)
epoch:5,auc:0.8346614436341048,last_auc:0.8345791491650397,dalta:8.229446906504734e-05
(67330, 1129)
(72412, 1129)
stop at epoch:6!


StackingClassifier(estimators=[('gbc',
                                GradientBoostingClassifier(max_depth=5,
                                                           n_estimators=50)),
                               ('hgbc',
                                HistGradientBoostingClassifier(max_depth=5)),
                               ('xgbc',
                                XGBClassifier(base_score=None, booster=None,
                                              callbacks=None,
                                              colsample_bylevel=None,
                                              colsample_bynode=None,
                                              colsample_bytree=None,
                                              early_stopping_rounds=None,
                                              enable_categorical=False,
                                              eval_metric='auc'...
                                              max_cat_to_onehot=None,
                                              max_delta_step=None, max_depth=6,
                                              max_leaves=None,
                                              min_child_weight=None,
                                              missing=nan,
                                              monotone_constraints=None,
                                              n_estimators=100, n_jobs=None,
                                              num_parallel_tree=None,
                                              predictor=None, random_state=None,
                                              reg_alpha=None, reg_lambda=None, ...)),
                               ('cbc',
                                <catboost.core.CatBoostClassifier object at 0x1545798d0>)],
                   final_estimator=LogisticRegression())

In [8]:
# save model
with open("./models/rankB_stack_pesudo_model.pkl", "wb") as file:
    pickle.dump(model,file=file)